# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - для построения модели кредитного скоринга проверить гипотезы:
1. Имеется зависимость между наличием детей и возвратом кредита в срок.
2. Семейное положениевлияет на возврат кредита в срок.
3. Уровень дохода влияет на возврат кредита в срок.
4. Цель кредита влияет на его возврат в срок.

**Ход исследования**
Статистика о платежеспособности клиентов получена от банка. О качестве данных ничего не известно. Поэтому перед проверкой гипотез будет проведен обзор данных.
Данные будут проверены на наличие на ошибок, будет оценено их влияние на исследование. Затем, на этапе предобработки будут исправлены самые критичные ошибки данных.
Таким образом, исследование пройдёт в три этапа:
1. Обзор данных.
2. Предобработка данных.
3. Проверка гипотез.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Импортируем один из наших главных инструментов - библиотеку pandas. Для прочтения файла используем конструкцию try-except, что позволяет нам прочитать файл как в тренажере, так и на локальном компьютере.

In [1]:
import pandas as pd #импорт библиотеки pandas
try: #используем конструкцию try-except чтобы прочитать файл
    df = pd.read_csv('C:/data_folder/data.csv') #пробуем прочитать файл на локальном компьютере
except:
    df = pd.read_csv('/datasets/data.csv') #читаем файл на сервере

Выводим на экран первые 15 строк таблицы, чтобы познакомиться с данными

In [2]:
display(df.head(10)) #используем метод head, чтобы не выводить весь датафрейм целиком

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Изучение первых строк таблицы позволяет сделать выводы о неидеальности данных: отрицательное количество дней трудового стажа, разный регистр в столбце возраст, возможно и другие. Посмотрим общую информацию о таблице методом info():

In [3]:
df.info()# получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице есть пропуски. Попробуем применить также метод pandas describe, который даст нам общую информацию о дата фрейме на языке описательной статистики:

In [4]:
df.describe()# получение статистической информации о числовых столбцах

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


До предобработки данных пока сложно сделать выводы, однако уже кажется, что портрет среднего заемщика: гражданин 43 лет, с ребенком (ну не с половиной же ребенка ;) ), образование среднее, живет в гражданском браке, не имеет долгов. Посмотрим потом как изменятся показания после предобработки и сравним. Кроме того блгодаря методу describe мы увидели, что есть слишком большие значения дней трудоустройства (max 401755, что примерно 1100 лет), а значит с этим вопросом так же необходимо разобраться во время предобработки данных.  
Еще мы увидели что минимальный возраст заемщика 0 лет, что тоже выглядит как ошибка выгрузки.

**Вывод**
Итак, в таблице двенадцать столбцов. Типы данных столбцов зависят от содержимого: вполне справделиво текстовые столбцы - education, family_status, gender, income_type, purpose типа object. Остальные целочисленных(int64) и вещественных(float64) типов.

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

С названием колонок все в порядке, однако для собственного удобства хочется в разделе предобработки данных заменить название столбца dob_years на age, так проще для восприятия. 


Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


## Шаг 2. Предобработка данных

### Обработка пропусков

Как уже было упомянуто, прежде чем приступить к обработке пропусков, заменим название столбца, которое мне было бы проще воспринимать как age, вместо dob_years:

In [5]:
df = df.rename(columns={'dob_years':'age'}) #переименовываю неугодный мне столбец

In [6]:
df.columns #проверяю сработало ли переименование вызвом перечня названий столбцов

Index(['children', 'days_employed', 'age', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Из Шага 1 информации и датафрейме, мы узнали, что у нас есть пропуски в столбцах, посмотрим количество пропусков в каждом столбце:

In [7]:
df.isna().sum() #считаем пропуски в каждом столбце последовательно применяя методы

children               0
days_employed       2174
age                    0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Не все пропущенные значения влияют на результат, в нашем случае пропуски в количестве отработанных дней и в месячном доходе составляют около 10% от выборки, однако ситуация усугубится, если мы удалим эти строки, а среди них были должники. Посмотрим сначала сколько у нас должников:

In [8]:
display(df['debt'].value_counts()) #смотрим какие значения в столбце должников и количество

0    19784
1     1741
Name: debt, dtype: int64

Теперь с помощью логической индексации проверим количество пересечений строк, где есть пропуски в опыте работы и месячном доходе и наличии долга:

In [9]:
display(df[df['days_employed'].isna() & df['debt'] == 1])

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
181,0,NaN,26,среднее,1,гражданский брак,1,F,компаньон,1,NaN,покупка жилья для семьи
247,1,NaN,60,высшее,0,женат / замужем,0,F,пенсионер,1,NaN,заняться высшим образованием
278,1,NaN,23,Среднее,1,гражданский брак,1,F,сотрудник,1,NaN,автомобиль
312,1,NaN,33,среднее,1,гражданский брак,1,M,сотрудник,1,NaN,покупка жилья для сдачи
...,...,...,...,...,...,...,...,...,...,...,...,...
20592,3,NaN,35,среднее,1,женат / замужем,0,F,сотрудник,1,NaN,получение дополнительного образования
20646,1,NaN,50,среднее,1,гражданский брак,1,F,компаньон,1,NaN,строительство собственной недвижимости
20917,0,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,1,NaN,строительство собственной недвижимости
21271,2,NaN,42,среднее,1,гражданский брак,1,M,сотрудник,1,NaN,операции со своей недвижимостью


ОГО! У целых 170 должников отсутсвуют данные об опыте работы и месячном доходе. Так как это большая доля от наших должников, мы не можем удалить пропуски, потому что они могут искозить результаты проверки гипотез. В связи с этим нам необходимо заполнить пропуски. Думаю, что в данном случае подойдет быстрый способ - найдем медианную месячную зарплату по выборке, а также медианный стаж в соответсвии с типом занятости. Но прежде чем заняться вычислениями, необходимо привести столбцы с данными об опыте работы и доходах в порядок, так как в них явно есть ошибки.

Сначала приведем в порядок столбец с опытом работы в днях. Во-первых там есть отрицательные значения. Возможно они появились из-за технической ошибки, а может свою роль сыграл человеческий фактор, к сожалению мы не можем уточнить у разработчиков, поэтому раюотаем с тем что есть. Для начала возьмем модуль каждого значения, чтобы избавиться от отрицательных:

In [10]:
from math import fabs #импортировали библиотеку, чтобы использовать функцию взятия модуля
df['days_employed'] = df['days_employed'].apply(fabs) #перезаписали значения столбца об опыте работы модулями старых значений
display(df.sort_values('days_employed').head()) #сортировка от минимальных, чтобы убедиться, что взятие по модулю сработало и впереди только положительные числа

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17437,1,24.141633,31,среднее,1,женат / замужем,0,F,сотрудник,1,166952.415427,высшее образование
8336,0,24.240695,32,высшее,0,Не женат / не замужем,4,M,сотрудник,0,124115.373655,получение дополнительного образования
6157,2,30.195337,47,среднее,1,гражданский брак,1,M,компаньон,0,231461.185606,свадьба
9683,0,33.520665,43,среднее,1,Не женат / не замужем,4,M,сотрудник,1,128555.897209,приобретение автомобиля
2127,1,34.701045,31,высшее,0,женат / замужем,0,F,компаньон,0,90557.994311,получение образования


Столбец с детьми также возьмем по модулю, так как функция describe показала нам, что минимальное количество детей -1: не годится!

In [11]:
df['children'] = df['children'].apply(fabs) #проделываем тоже самое с количеством детей
df.describe() #смотрим поменялось ли что то и высматриваем что еще может быть не так

,children,days_employed,age,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.543275,66914.728907,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.379876,139030.880527,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Мы помним, что у рабочего стажа в днях было выбросы - слишком большие значения. Возможно данные были выгружены в часах. Чтобы посмотреть общие данные о столбце, после того как мы избавились от отрицательных значений:

In [12]:
df['days_employed'].describe() #описательная статистка!

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

Для того, чтобы перевести значения, выраженные не в днях, а в часах, проверим чтобы стаж работы в днях не превышал возраста заемщика в днях. Если стаж превышает возраст, значит он выражен в часах и мы делим значение стажа на 24. Но насколько мы выяснили из функции describe  - есть заемщики, чей возраст равен 0, а это ошибка. Таким образом надо заменить 0 на среднйи возраст, характерный для соответсвующей категории заяемщика. Группировку заемщиков  в исследовании будем производить по типу занятости, потому что это наилучший вариант категоризации данных в случае возраста, опыта работы, ежемесячного дохода :

In [13]:
median_age = df.groupby('income_type')['age'].median() #посчтитали медианные значения возраста для каждой категории занятости заемщиков
display(median_age)
df1 = df[df['age'] < 18] #заодно проверим есть ли еще заемщики младше 18: для этого сохраним строки датафрейма, где возраст меньше 18 в другую переменную
display(df1['age'].unique())#посмотрим уникальные значения в столбце age методом unique
display(df1['income_type'].unique())# а это на всякий случай, чтобы понимать у каких категорий заемщиков ошибка в возрасте и проверить теорию возникновения ошибки
df.loc[(df['age'] == 0) & (df['income_type'] == 'пенсионер'), 'age'] = median_age[4]
df.loc[(df['age'] == 0) & (df['income_type'] == 'сотрудник'), 'age'] = median_age[6]
df.loc[(df['age'] == 0) & (df['income_type'] == 'компаньон'), 'age'] = median_age[3]
df.loc[(df['age'] == 0) & (df['income_type'] == 'госслужащий'), 'age'] = median_age[2]


income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: age, dtype: float64

array([0])

array(['пенсионер', 'сотрудник', 'компаньон', 'госслужащий'], dtype=object)

In [14]:
display(df[df['age'] == 0]) #проверяем, что все сработало 

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Теперь, когда мы решили проблемы с возрастом заемщиков, мы можем исправить проблему со стажем отталкиваясь от возраста: невозможно, чтобы стаж был больше возраста заемщика, соответсвенно поставим такое условие и изменим значения ячеек:

In [15]:
df.loc[df['days_employed'] > df['age'] * 365, 'days_employed'] = df.loc[df['days_employed'] > df ['age'] * 365, 'days_employed'] / 24
display(df['days_employed'].describe()) #посмотреть что изменилось


count    19351.000000
mean      4641.641176
std       5355.964289
min         24.141633
25%        927.009265
50%       2194.220567
75%       5537.882441
max      18388.949901
Name: days_employed, dtype: float64

Теперь можно приступать к поиску медианных значений в разрезе типа занятости:

In [16]:
median_days_employed = df.groupby('income_type')['days_employed'].median() #посчитали медианы для каждого типа занятости по опыту работы, чтоюы 
display(median_days_employed)
median_total_income = df.groupby('income_type')['total_income'].median() #посчитали медианы доходов
display(median_total_income)

income_type
безработный        15267.235531
в декрете           3296.759962
госслужащий         2689.368353
компаньон           1547.382223
пенсионер          15217.221094
предприниматель      520.848083
сотрудник           1574.202821
студент              578.751554
Name: days_employed, dtype: float64

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Медианы по типам занятости посчитали - самое время заполнить пропуски. Чтобы не писать на каждый тип занятости логическую индексацию (первая закомменченная строчка, если по каждому так прописать, то все заполнится), создадим таблицу, которая будет содержать перечень типов занятости и их номера, номера соответсвуют индексу типа занятости в переменных с медианами. Затем выполним объединение таблиц, чтобы в таблице df каждому типу занятости соответствовал ее айди и после этого реализуем цикл, который в зависимости от значения i будет заполнять пропуски в таблице.  
В данном блоке я хотела продемонстрировать умение создавать новый датафрейм, знание циклов, построенных по индексу и метод merge из теории, однако с методом merge у меня задвоились столбцы и выполнилось некорректно (эту попытку я закомментировала в коде) и я пошла обходным путем, благо мне нужно было добавить только 1 столбец.

In [17]:
#df.loc[df['income_type'] == 'пенсионер', 'days_employed'] = df.loc[df['income_type'] == 'пенсионер', 'days_employed'].fillna(median_days_employed[4])
#df.info()
spisok = [['безработный', 0],['в декрете', 1],['госслужащий', 2],['компаньон', 3],['пенсионер', 4],['предприниматель', 5],['сотрудник', 6],['студент', 7]]
income_dict = pd.DataFrame(spisok, columns = ['income_type','income_type_id']) #тут я создаю датафрейм где каждому типу занятости соответсвует индекс этого типа занятости из переменных с медианами 
income_dict.info() #смотрю получилось ли создать датафрейм и типы данных


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
income_type       8 non-null object
income_type_id    8 non-null int64
dtypes: int64(1), object(1)
memory usage: 256.0+ bytes


Далее объединяю дата фреймы, однако видимо неправильно записываю функцию merge(), потому что при ее использовании новый столбец - множится, но я нахожу изящный способ добавить только один столбец:

In [18]:
#df = pd.merge(left = df, right = income_dict, on = 'income_type') # не сработало 
df['income_type_id'] = df['income_type'].map(income_dict.set_index('income_type')['income_type_id']) #а так получилось!
df.info() #проверяем
#display(df['income_type_id'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null float64
days_employed       19351 non-null float64
age                 21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
income_type_id      21525 non-null int64
dtypes: float64(4), int64(4), object(5)
memory usage: 2.1+ MB


Теперь можно реализовать цикл по индексам и не создавая лишних строк цикла заменить все пропущенные значения:

In [19]:
for i in range(len(spisok)): #цикл по индексам, можно было написать range(8), но я надеюсь мне зачтется использование функций
    df.loc[df['income_type_id'] == i, 'days_employed'] = df.loc[df['income_type_id'] == i, 'days_employed'].fillna(median_days_employed[i])
    df.loc[df['income_type_id'] == i, 'total_income'] = df.loc[df['income_type_id'] == i, 'total_income'].fillna(median_total_income[i])
df.info() #проверяем остались ли пропуски
df.describe() #чтобы было понимание, что ничего не вышло из под контроля

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null float64
days_employed       21525 non-null float64
age                 21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
income_type_id      21525 non-null int64
dtypes: float64(4), int64(4), object(5)
memory usage: 2.1+ MB


,children,days_employed,age,education_id,family_status_id,debt,total_income,income_type_id
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04,21525.000000
mean,0.543275,4600.535795,43.496167,0.817236,0.972544,0.080883,1.652253e+05,4.661045
std,1.379876,5354.354167,12.231538,0.548138,1.420324,0.272661,9.804367e+04,1.462582
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04,0.000000
25%,0.000000,1025.549623,34.000000,1.000000,0.000000,0.000000,1.077982e+05,3.000000
50%,0.000000,1993.522017,43.000000,1.000000,0.000000,0.000000,1.425944e+05,6.000000
75%,1.000000,5347.024506,53.000000,1.000000,1.000000,0.000000,1.955499e+05,6.000000
max,20.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,2.265604e+06,7.000000


**Вывод**

Кажется мы привели данные в порядок: взяли по модулю отрицательные значения, исправили проблемы с возрастом равным нулю, привели в порядок стаж работы, заполнили пропуски, а главное все в разрезе типа занятости! Можно двигаться дальше!

### Замена типа данных

Еще раз посмотрим какие типы данных у столбцов:

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null float64
days_employed       21525 non-null float64
age                 21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
income_type_id      21525 non-null int64
dtypes: float64(4), int64(4), object(5)
memory usage: 2.1+ MB


Количество детей - определнно должно быть целым числом, ровно так же как количество отработанных дней, да и в доходах, нас мало интересуют копейки и будет удобно свести все к рублям. Чтобы привести вещественные числа к целочисленному типу ввоспользуемся функцией astype():

In [21]:
df['children'] = df['children'].astype('int')
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

df.info() #всегда проверяем результат!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
age                 21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
income_type_id      21525 non-null int64
dtypes: float64(1), int64(7), object(5)
memory usage: 2.1+ MB


**Вывод**

Удалось привести все числовые столбцы к целочисленному типу int64 - красота!

### Обработка дубликатов

Для того, чтобы результат анализа был более точным, необходимо избавится от дубликатов. Проведем поиск полных дубликатов с помощью duplicated():

In [22]:
print('Найдено дубликатов с помощью метода duplicated:', df.duplicated().sum())

Найдено дубликатов с помощью метода duplicated: 54


54 дубликата, чтож для таблицы из 21525 наблюдений это не так много, но избавится от них займет всего минуту, не будем засорять данные и очистим:

In [23]:
df = df.drop_duplicates().reset_index(drop = True) #удаляем дубли с помощью соответсвующего метода
print('Найдено дубликатов с помощью метода duplicated:', df.duplicated().sum()) #проверяем что получилось

Найдено дубликатов с помощью метода duplicated: 0


Вроде от дубликатов избавились, однако при просмотре таблицы можно было заметить, что у столбца образование скачет регистр, поэтому приведем все значения этого столбца к нижнему регистру:

In [24]:
df['education'] = df['education'].str.lower() 

Проверим уникальные значения, чтобы знать, что все сработало, заодно првоерим уникальные значения столбцов, с которыми мы еще не работали и не замечали проблем, вдруг там тоже есть косяки с регистром:

In [25]:
print(df['education'].unique())
print(df['family_status'].unique())
print(df['purpose'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильн

Узнаем всплыли ли новые дубликаты при изменении регистра:

In [26]:
print('Найдено дубликатов с помощью метода duplicated:', df.duplicated().sum())

Найдено дубликатов с помощью метода duplicated: 17


Эх все таки надо было сначала привести в порядок регистр, но ничего:

In [27]:
df = df.drop_duplicates().reset_index(drop = True)
print('Найдено дубликатов с помощью метода duplicated:', df.duplicated().sum())

Найдено дубликатов с помощью метода duplicated: 0


**Вывод**

Избавились от явных дубликатов, а также от неявных, сведя текстовые значения к единому регистру, теперь в данных нет дубликатов. Однако во время просмотра уникальных значений столбца с целью взятия кредита было обнаружено, что одинаковые по факту цели записаны по разному, с этим неудобно работать, поэтому перейдем к следующему шагу: приведению целей кредита в понятный, удобный и единый вид путем лемматизации.

### Лемматизация

Мы уже получили уникальные значения столбца purpose(цель кредита), для того чтобы выделить основные цели, соберем все значения в единую строчку и проведем лемматизацию. 

In [28]:
from pymystem3 import Mystem #импорт нужной библиотеки
m = Mystem () #зачем каждый раз писать Mystem, если m - короче, а короче, значит быстрее!
purposes = ' '.join(df['purpose'].unique()) #сохраняет в переменную строку созданную из списка уникальных значений столбца purpose
#display(purposes) #проверяем, что получилось
lemmas = m.lemmatize(purposes) #присваиваем новой переменной lemmas значение лемматизированной переменной purposes
print(lemmas)

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование', ' ', 'сделка', ' ', 'с', ' ', 'подержать

Теперь необходимо подсчитать количество упоминаний лемматизированных слов, чтобы определить наши будущие цели кредита. Для этого используем специальный контейнер Counter из модуля collections:

In [29]:
from collections import Counter #
print(Counter(lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


Изучив результаты можно выделить несколько категорий:  
**Недвижимость** - сюда мы отнесем все что касается, недвижимости и жилья, а так же операций связанных с ними, в том числе покупкуб строительство, ремонт и прочие  
**Образование** - сюда отнесем все что связано с получением образования, в том числе повышения квалификации, высшее образование, вторые дипломы и др.  
**Автомобиль** - это категория автокредитов  
**Свадьба** - эта категория говорит сама за себя.  
Хм, ни одного кредита на открытие бизнеса.  

Теперь когда категории определены, надо присвоить каждому заемщику цель кредита в соответсвии с нашими категориями, для этого сначала напишем функцию, которая на входе будет получать значение из ячейки и проверять к какой из определенных категорий можно отнести значение из ячейки.

In [30]:
def find_purpose_category(purpose):
    """Функция возвращает категорию цели кредита исходя из значения цели кредита"""
    lemmas = m.lemmatize(purpose) #функция лемматизирует значение на входе
    if 'образование' in lemmas: #а потом мы проверяем есть ли наши лемматизированные категории в результате лемматизации значения на входе
        return 'образование'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'свадьба' in lemmas:
        return 'свадьба'
    return 'недвижимость' #недвижимость ставим последней, но можно было написать условие if 'недвижимость' or 'жилье' in lemmas

Добавим столбец с применением функции:

In [31]:
df['purpose_category'] = df['purpose'].apply(find_purpose_category) #создаем новый столбец с нашими категориями
display(df.head()) #смотрим и радуемся, что все получилось как задумано

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_id,purpose_category
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,6,недвижимость
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,6,автомобиль
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,6,недвижимость
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,6,образование
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4,свадьба


**Вывод**

Нам удалось получить вполне внятные категории целей кредита, вместо многочисленных вариантов записи одних и тех же целей. Благодаря новому столбцу с категориями целей кредита мы сможем группировать наших заемищиков по целям кредита.

### Категоризация данных

 Для ответов на гипотезы осталось категаризировать данные по уровню дохода. Для простоты и удобства поделим доход на категории в соответсвии с процентилями. Для этого воспользуемся библиотекой numpy:

In [32]:
import numpy as np #подключаем библиотеку, чтобы использовать метод поиска процентиля
q1 = np.percentile(df['total_income'], 25)
q2 = np.percentile(df['total_income'], 50)
q3 = np.percentile(df['total_income'], 75)
print(q1,q2,q3) #вывод получившихся процентилей, можно было воспользоваться describe, но мы уже им пользовались, а проект направлен на то, чтобы показать широту освоенной теории

107623.0 142594.0 195820.25


Теперь, когда нам известны процентили можно категорезировать данные. У нас будет 4 группы дохода и чтобы никого не обижать(в первую очередь судя по всему себя), они будут называться 1,2,3,4.

In [33]:
def income_type(income):
    """Определяем уровень дохода и присваиваем соответсвующую категорию"""
    if income <= q1:
        return '4'
    if q1 < income <= q2:
        return '3'
    if q2 < income <= q3:
        return '2'
    return '1'

Теперь применим написанную функцию к столбцу общего дохода нашего чудесного датафрейма:

In [34]:
df['total_income_type'] = df['total_income'].apply(income_type)
display(df.head())

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_id,purpose_category,total_income_type
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,6,недвижимость,1
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,6,автомобиль,3
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,6,недвижимость,2
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,6,образование,1
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4,свадьба,2


**Вывод**

Все готово к проверке гипотез.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Лучше всего линейнгую зависимость между двумя массивами перменных показывает коэффициент корреляции:

In [35]:
df['children'].corr(df['debt'])

0.01758315233290855

Корреляция очень маленькая, однако все равно попробуем составить таблицу, сгруппированную по наличию детей и посмотрим долю должников внутри каждой категории:

In [36]:
report_children = df.groupby('children').agg({'debt': ['count', 'sum','mean']})
display(report_children)

debt                
          count   sum      mean
children                       
0         14091  1063  0.075438
1          4855   445  0.091658
2          2052   194  0.094542
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000
20           76     8  0.105263

Очень смущает количество детей 20, если до проверки зависимости казалось, что ну есть у кого то 20 детей и ладно, то учитывая что доля должников у людей с 20ю детьми близка к доле должников с 2 детьми, наличие 20ти детей воспринимается как опечатка при попытке указать 2х детей, исправим:

In [37]:
df.loc[df['children'] == 20, 'children'] = 2
print(df['children'].corr(df['debt']))
report_children = df.groupby('children').agg({'debt': ['count', 'sum','mean']})
display(report_children)

0.02468601160325064


debt                
          count   sum      mean
children                       
0         14091  1063  0.075438
1          4855   445  0.091658
2          2128   202  0.094925
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000

Уровень корреляции подрос, однако до сих пор слишком маленький, чтобы утверждать, что между наличием детей и возвратом кредита в срок есть связь. Поразительно, что у заемщиков с 5ю детьми нет долгов (большая семья - большая ответсвенность?), однако доли должников среди тех у кого 1-4 детей примерно равны, чтож, чтобы ответить на вопрос укрупим категоризацию до есть дети и нет детей и посмотрим долю должников:

In [38]:
print(df[df['children'] == 0].agg({'debt': 'mean'}))
print(df[df['children'] > 0].agg({'debt': 'mean'}))


debt    0.075438
dtype: float64
debt    0.092082
dtype: float64


**Вывод**

Доля должников среди бездетных 7,5%, а должников среди заемщиков с детьми - 9%. Доля должников во всей выборке 8,8%, таким образом бездетные чаще, чем в среднем по выборке возвращают кредит в срок, а семьи с детьми - чуть реже.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [39]:
df['family_status_id'].corr(df['debt'])
report_family = df.groupby('family_status').agg({'debt': ['count', 'sum','mean']})
report_family.columns =['всего','должники','% невозврата']
display(report_family.sort_values(by = '% невозврата'))

,всего,должники,% невозврата
family_status,,,
вдовец / вдова,959,63,0.065693
в разводе,1195,85,0.071130
женат / замужем,12339,931,0.075452
гражданский брак,4151,388,0.093471
Не женат / не замужем,2810,274,0.097509


**Вывод**

Поразительно, но люди в гражданском браке в среднем становятся должниками почти так же часто как и неженатые. Отталкиваясь от средней задолженности по выборке (напоминаю 8,8%), можно сказать с уверенностью, что овдовевшие заемщики - самые сознательные! Но тут как с первой гипотезой: все кто перешагнул барьер в 8,8% уже рискуют задолжать чаще, чем те, у кого процент невозврата меньше.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:
df['total_income'].corr(df['debt'])

-0.012594884662512325

Я очень люблю корреляции, но видимо они не работают, когда вметсо размера долга, указана просто единичка. Однако уже плюс - корреляция отрицательная, то есть чем больше доход, тем реже долги (вот это инсайт!)! Посмотрим что будет происходить в разрезе категорий:

In [41]:
report_income = df.groupby('total_income_type').agg({'debt': ['count', 'sum','mean']})
report_income.columns =['всего','должники','% невозврата']
display(report_income.sort_values(by = '% невозврата'))

,всего,должники,% невозврата
total_income_type,,,
1,5364,383,0.071402
4,5364,427,0.079605
2,5247,448,0.085382
3,5479,483,0.088155


**Вывод**

Теперь понятно, почему корреляция такая крохотенюшка: 4 категоря доходов - самая низкая! и при этом эти ребята отдают кредит чаще, чем более зажиточные 2 и 3!. Я бы сказала, что уровень дохода не показатель, разве что самые платежеспособные отдают долги лучше всех, но и они могут задолжать. 


- Как разные цели кредита влияют на его возврат в срок?

In [42]:
report_purposes = df.groupby('purpose_category').agg({'debt': ['count', 'sum','mean']})
display(report_purposes)

debt               
                  count  sum      mean
purpose_category                      
автомобиль         4306  403  0.093590
недвижимость      10811  782  0.072334
образование        4013  370  0.092200
свадьба            2324  186  0.080034

**Вывод**

Ну что тут можно сказать, покупка недвижимости-дисциплинирует, а вот после получения образования надо еще успеть заработать, чтобы отдать долги по кредиту.

Попробуем сделать общие выводы, составив сводные таблицы, продемонстриуруем умения, а также надеюсь получим какие бы то ни было дополнительные инсайты.

In [43]:
df_pivot_children = df.pivot_table( index = 'children', columns = 'income_type', values = 'debt', aggfunc = 'mean')
display(df_pivot_children)

income_type,безработный,в декрете,госслужащий,компаньон,пенсионер,предприниматель,сотрудник,студент
children,,,,,,,,
0,0.0,NaN,0.068129,0.072043,0.056410,0.0,0.088226,0.0
1,1.0,NaN,0.053073,0.081102,0.049470,NaN,0.104955,NaN
2,NaN,1.0,0.031746,0.070780,0.103448,NaN,0.112666,NaN
3,NaN,NaN,0.055556,0.063291,0.166667,NaN,0.090909,NaN
4,NaN,NaN,0.000000,0.000000,0.000000,NaN,0.129032,NaN
5,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,NaN


Стало интересно, как так вышло, что у нас заемщикив  декрете в 100% случаев не отдают долги(

In [44]:
display(df[df['income_type'] == 'в декрете'])

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_id,purpose_category,total_income_type
20778,2,3296,39.0,среднее,1,женат / замужем,0,F,в декрете,1,53829,автомобиль,1,автомобиль,4


Теперт понятно (всего один такой человек, не показательно), стоит взять на заметку что человеку в декрете с низким уровнем дохода не стоит давать кредит на автомобиль)

In [45]:
df_pivot_family = df.pivot_table( index = 'family_status', columns = 'income_type', values = 'debt', aggfunc = 'mean')
display(df_pivot_family)

income_type,безработный,в декрете,госслужащий,компаньон,пенсионер,предприниматель,сотрудник,студент
family_status,,,,,,,,
Не женат / не замужем,NaN,NaN,0.073171,0.089330,0.045584,NaN,0.116935,0.0
в разводе,NaN,NaN,0.049383,0.029412,0.059361,NaN,0.096308,NaN
вдовец / вдова,NaN,NaN,0.044444,0.092784,0.068901,NaN,0.053571,NaN
гражданский брак,0.0,NaN,0.068702,0.093966,0.055470,0.0,0.107319,NaN
женат / замужем,1.0,1.0,0.055249,0.066390,0.054993,0.0,0.088617,NaN


Самое показательное в плане надежности кредиторов - оказывается их тип занятости, из сводных таблиц сразу видно, что безработные и заемщики в декрете - не надежны. А предприниматели - лучшие клиенты банков! Проверим гипотезу сгруппировав по типу занятости:

In [46]:
report_income_type = df.groupby('income_type').agg({'debt': ['count', 'sum','mean']})
report_income_type.columns =['всего','должники','% невозврата']
display(report_income_type.sort_values(by = '% невозврата'))

,всего,должники,% невозврата
income_type,,,
предприниматель,2,0,0.000000
студент,1,0,0.000000
пенсионер,3829,216,0.056412
госслужащий,1457,86,0.059025
компаньон,5078,376,0.074045
сотрудник,11084,1061,0.095724
безработный,2,1,0.500000
в декрете,1,1,1.000000


К сожалению слишком мало студентов, предпринимателей, безработных и в декрете в выборке, чтобы утверждать, что их тип занятости напрямую влияет на возврат кредита в срок :( Зато можно не бояться давать кредиты пенсионерам, госслужащим и компаньонам) Пройдемся дальше по общим выводам:

In [47]:
df_pivot_income = df.pivot_table( index = 'total_income_type', columns = 'income_type', values = 'debt', aggfunc = 'mean')
display(df_pivot_income)

income_type,безработный,в декрете,госслужащий,компаньон,пенсионер,предприниматель,сотрудник,студент
total_income_type,,,,,,,,
1,0.0,NaN,0.048718,0.063622,0.060504,0.0,0.083169,NaN
2,NaN,NaN,0.048035,0.079011,0.057489,NaN,0.104049,NaN
3,NaN,NaN,0.081395,0.079327,0.065004,NaN,0.098898,NaN
4,1.0,1.0,0.068376,0.082777,0.047586,NaN,0.096051,0.0


In [48]:
df_pivot_purpose = df.pivot_table( index = 'purpose_category', columns = 'income_type', values = 'debt', aggfunc = 'mean')
display(df_pivot_purpose)

income_type,безработный,в декрете,госслужащий,компаньон,пенсионер,предприниматель,сотрудник,студент
purpose_category,,,,,,,,
автомобиль,NaN,1.0,0.076923,0.080798,0.064151,NaN,0.112339,NaN
недвижимость,0.5,NaN,0.047745,0.065567,0.048652,0.0,0.086554,0.0
образование,NaN,NaN,0.081395,0.075472,0.066574,NaN,0.110096,NaN
свадьба,NaN,NaN,0.044025,0.099048,0.059242,0.0,0.083813,NaN


На самом деле я считаю выборку слишком неоднородной и слишком маленькой, чтобы делать какие то 100% утверждения, однако точно можно сказать, что большинство должников - это сотрудники  с детьми, не зависимо от уровня дохода, берущие кредит на автомобили/образование. Дальнешие рекомендации для составления можели кредитного скоринга - учитывать процент должников больше среднего 8,8% как те кто склонны чаще не вернуть кредит в срок, а тех у кого процент меньше 8,8%  более склонных отдать кредит во время.  
Поиск зависимости без категоризации по типу занятости не так репрезентативен, как в разрезе типа занятости и другого фактора.
